In [30]:
import pandas as pd
import os
from sylk_parser import SylkParser
from datetime import datetime, timedelta
import cx_Oracle
import re
import xlsxwriter


def FETCH_DATA():
    # Define connection details
    dsn = cx_Oracle.makedsn(
        host = "192.168.1.242",      # Replace with your host IP or domain
        port = 1526,                # Replace with your port
        service_name = "sperpdb"  # Replace with your service name
    )

    # Establish the connection
    connection = cx_Oracle.connect(
        user = "spselect",         # Replace with your username
        password = "select",     # Replace with your password
        dsn = dsn
    )
    query = """
        SELECT CST_REFE_NO, SC_NO, ORD_DATE, VEN_DLV_DATE, DLV_DATE
        FROM V_SCH0200Q_ORD 
        WHERE CST_PART_NO LIKE 'EB%'
        AND ORD_CST_NO = 'D09200'
    """
    df = pd.read_sql_query(query, connection)
    connection.close()
    return df
    

List = FETCH_DATA()
List = List.drop_duplicates(subset='SC_NO')


C:\Users\sales6\AppData\Local\Temp\ipykernel_17368\1435926678.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


In [31]:
List = List.rename(columns={
    "CST_REFE_NO": "客戶PO號",
    "ORD_DATE": "訂單日期",
    "VEN_DLV_DATE": "生管交期",
    "DLV_DATE": "訂單交期"
})


Order_List = List[~List["客戶PO號"].str.contains("庫存")]
Stock_List = List[List["客戶PO號"].str.contains("庫存")]

Order_List["下單至交貨天數"] = (Order_List["訂單交期"] - Order_List["訂單日期"]).dt.days

Order_List = Order_List.sort_values(by="訂單交期").reset_index(drop=True)
Order_List.index = Order_List.index + 1
Stock_List = Stock_List.sort_values(by="SC_NO").reset_index(drop=True)

C:\Users\sales6\AppData\Local\Temp\ipykernel_17368\1120420812.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Order_List["下單至交貨天數"] = (Order_List["訂單交期"] - Order_List["訂單日期"]).dt.days


In [32]:

# === STEP 1: Create weekly ranges ===
start_date = datetime(2025, 5, 1)
end_date = datetime(2025, 12, 31)

current_monday = start_date - timedelta(days=start_date.weekday())

weeks = []

while current_monday <= end_date:
    week_number = current_monday.isocalendar()[1]
    week_start = current_monday
    week_end = current_monday + timedelta(days=6)
    
    if week_end >= start_date:
        weeks.append({
            "第_周": week_number,
            "訂單交期": f"{week_start.strftime('%m/%d')}~{week_end.strftime('%m/%d')}",
            "Start": week_start,
            "End": week_end
        })
    
    current_monday += timedelta(weeks=1)

weeks_df = pd.DataFrame(weeks)

# === STEP 2: Assign each Order to a week ===

def get_week_range(date):
    for _, row in weeks_df.iterrows():
        if row["Start"] <= date <= row["End"]:
            return row["訂單交期"]
    return None

Order_List["出貨周"] = Order_List["訂單交期"].apply(get_week_range)

# === STEP 3: Group by week and get SC_NO lists ===
Order_List["SC(PO)"] = Order_List["SC_NO"].astype(str) + "(" + Order_List["客戶PO號"].astype(str) + ")"
Order_List["櫃數編號"] = "第" + Order_List.index.astype(str) + "櫃"

# Step 4: Group by '出貨周' separately
grouped_sc = Order_List.groupby("出貨周")["SC(PO)"].apply(list).reset_index()
grouped_index = Order_List.groupby("出貨周")["櫃數編號"].apply(list).reset_index()

# Step 5: Merge both with weeks_df and expand into columns

# For SC(PO)
weeks_df_sc = weeks_df.merge(grouped_sc, how="left", left_on="訂單交期", right_on="出貨周")
weeks_df_sc = weeks_df_sc[["第_周", "訂單交期", "SC(PO)"]]
sc_df = weeks_df_sc["SC(PO)"].apply(pd.Series)
weeks_df_sc = pd.concat([weeks_df_sc[["第_周", "訂單交期"]], sc_df], axis=1)


In [33]:
for col in ["訂單日期", "生管交期", "訂單交期"]:
    for df in [Order_List, Stock_List]:
        df[col] = pd.to_datetime(df[col], errors='coerce').dt.strftime('%Y/%m/%d')


output_file = r"Z:\跨部門\共用資料夾\業務 to 包裝 包材及出貨資訊\D092\水泥板螺絲系列\D092水泥板螺絲對照表.xlsx"
with pd.ExcelWriter(output_file, engine="xlsxwriter") as writer:
    weeks_df_sc.to_excel(writer, index=False, sheet_name="依ERP訂單交期-每周")
    Order_List[["客戶PO號", "SC_NO", "訂單日期", "生管交期", "訂單交期", "下單至交貨天數"]].to_excel(writer, index=True, sheet_name="正式訂單(櫃數編號依出貨資料)")
    Stock_List[["SC_NO","客戶PO號", "生管交期"]].to_excel(writer, index=True, sheet_name="庫存單對應SC")